In [1]:
from pyspark.sql import SparkSession
from datetime import date

# Initialize SparkSession with Iceberg (HadoopCatalog)
spark = SparkSession.builder \
    .appName("IcebergCreateTable") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "/tmp/iceberg_warehouse") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2") \
    .getOrCreate()

In [2]:
data = [
    (1, "Trinity", date(2024, 1, 1)),
    (2, "Alex", date(2024, 2, 1)),
    (3, "Morpheus", date(2024, 3, 1)),
    (4, "Neo", date(2024, 4, 1)),
    (5, "Smith", date(2024, 5, 1)),
    (6, "Anderson", date(2024, 6, 1))
]

df = spark.createDataFrame(data, ["id", "name", "signup_date"])

In [3]:
# Ensure the database exists in the Iceberg catalog
spark.sql("CREATE DATABASE IF NOT EXISTS local.db")

DataFrame[]

In [4]:
# Write the DataFrame to an Iceberg-managed table
df.writeTo("local.db.sample_users") \
    .using("iceberg") \
    .tableProperty("format-version", "2") \
    .partitionedBy("signup_date") \
    .createOrReplace()